In [1]:
import json

json_filepath = '../Human_anno/overall_consistency.json'
with open(json_filepath,'r') as f:
    oc = json.load(f)

In [2]:
def rank_basescore(base_score, draw_ratio):
    # 计算最大值和最小值的差值
    score_range = max(base_score) - min(base_score)
    # 计算平序的阈值
    draw_gap = draw_ratio * score_range

    # 对列表进行排序并保留原始索引
    indexed_scores = list(enumerate(base_score))
    indexed_scores.sort(key=lambda x: x[1])

    # 处理平序
    ranks = [0] * len(base_score)
    current_rank = 1
    for i in range(len(indexed_scores)):
        if i > 0 and abs(indexed_scores[i][1] - indexed_scores[i - 1][1]) < draw_gap:
            ranks[indexed_scores[i][0]] = current_rank
        else:
            current_rank = i + 1
            ranks[indexed_scores[i][0]] = current_rank

    return ranks

In [3]:
models = ['gen2','lavie','pika','show1','videocrafter2']
vbench_res = {}
for model in models:
    path = '../baseline/overall_consistency/vbench_viclip_evaluation_results/results_overall_consistency_{}_eval_results.json'.format(model)
    with open(path,'r') as f:
        vbench_res[model] = json.load(f)

In [4]:
new_models = ['gen3','kling','gen2','lavie','pika','show1','videocrafter2']
for i in range(len(oc)):
    oc[i]['baseline_rank'] = {}
    for model in new_models:
        oc[i]['baseline_rank'][model] = 0


In [5]:
vbench_res['gen2']['overall_consistency'][1]
for i in range(len(oc)):
    base_score = [vbench_res[model]['overall_consistency'][1][i]['video_results'] for model in models]
    ranks = rank_basescore(base_score, 0.1)
    for j in range(len(models)):
        oc[i]['baseline_rank'] [models[j]] = ranks[j]

In [6]:
with open(json_filepath,'w') as f:
    json.dump(oc,f,indent=4)